# Implementing Support Enumeration in Python: Hints

## Exercises on NumPy arrays

First follow [the lecture](http://lectures.quantecon.org/py/numpy.html) on `NumPy`.

Import `NumPy`:

In [1]:
import numpy as np

Consider the following 1-dimensional array:

In [2]:
n = 5
a = np.arange(n)

In [3]:
a

array([0, 1, 2, 3, 4])

Obtain the value in the last entry:

Obtain the values in all entries except the last:

Assign -10 to the last entry:

Next consider the following 2-dimensional array:

In [4]:
m, n = 3, 3
b = np.arange(m*n).reshape(m, n)

In [5]:
b

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

Compute the matrix-vector mutiplication of `b` and `[1/3, 1/3, 1/3]`:

Let

In [6]:
I = [1, 2]
J = [0, 1]

We want to obtain the submatrix `b_IJ` of `b` with rows in `I` and columns in `J`.

Obtain the submatrix `b_I` of `b` with rows in `I`:

Obtain the submatrix `b_IJ` of `b_I` with columns in `J`:

Thus, we can obtain `b_IJ` from `b` by:

Assign 100 to the last column of `b`:

Assign -20 to all the entries except those in the last row and the last column:

## Example game

Consider the example from von Stengel (2007):
$$
A =
\begin{bmatrix}
3 & 3 \\
2 & 5 \\
0 & 6
\end{bmatrix},
\quad
B =
\begin{bmatrix}
3 & 2 \\
2 & 6 \\
3 & 1
\end{bmatrix}.
$$

It will be convenient to denote the actions by *indices*,
so let the action spaces of players 1 and 2 be
$$
M = \{0, 1, 2\}, \quad
N = \{0, 1\}.
$$
(Note that in Python, indices start with 0.)

Create NumPy arrays for the payoff matrices:

In [7]:
A = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
B = np.array([[3, 2],
              [2, 6],
              [3, 1]])

In [8]:
m, n = A.shape  # Numbers of actions of players 1 and 2, respectively
M = np.arange(m)
N = np.arange(n)

It will be more convenient to work with the [transpose](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.T.html) of matrix `B`:

In [9]:
B_T = B.T

In [10]:
B_T

array([[3, 2, 3],
       [2, 6, 1]])

## Support enumeration

For each $k = 1, \ldots, \min\{m, n\}$
and each support pair $(I, J)$ with $\lvert I\rvert = \lvert J\rvert = k$:

Solve the systems of linear equations:
$$
C \begin{pmatrix} y_J \\ u \end{pmatrix} = e
$$
and
$$
D \begin{pmatrix} x_I \\ v \end{pmatrix} = e
$$
with
$$
C =
 \begin{pmatrix}
 A_{IJ} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}, \quad
D =
 \begin{pmatrix}
 B'_{JI} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}, \quad
e = \begin{pmatrix}\mathbf{0} \\ 1\end{pmatrix},
$$
where
$A_{IJ}$ is the submatrix of $A$ given by rows $I$ and columns $J$,
$B'_{JI}$ is the submatrix of $B'$ given by rows $J$ and columns $I$, and
$\mathbf{0}$ and $\mathbf{1}$ are the $k$-dimensional vectors of zeros and ones,
respectively.

Then check that $x_i > 0$ for all $i \in I$ and $y_j > 0$ for all $j \in J$, and
that
$u \geq (A_J y_J)_i$ for all $i \notin I$ and
$v \geq (B'_I x_I)_j$ for all $j \notin J$,
where
$A_J$ is the submatrix of $A$ given by columns $J$ and
$B'_I$ is the submatrix of $B'$ given by columns $I$.

### Example: $I = \{0, 1\}$, $J = \{0, 1\}$

As an example, let $I = \{0, 1\} \subset M$ and $J = \{0, 1\} \subset N$,
and get the corresponding submatices of $A$ and $B'$.

In [11]:
k = 2
I = [0, 1]
J = [0, 1]

Submatrix $A_{IJ}$:

In [12]:
A_IJ = A[I, :][:, J]

In [13]:
A_IJ

array([[3, 3],
       [2, 5]])

Submatrix $B'_{JI}$:

In [14]:
B_T_JI = B_T[J, :][:, I]

In [15]:
B_T_JI

array([[3, 2],
       [2, 6]])

Construct the matrices $C$ and $D$ and the vector $e$,
and pass them the linear equation solver [`np.linalg.solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html).

First construct the vector $e = (0, \ldots, 0, 1)$:

In [16]:
e = np.zeros(k+1)  # Vector of zeros of length k+1
e[-1] = 1  # Assign 1 to the last entry of e

In [17]:
e

array([ 0.,  0.,  1.])

Here's how to construct
$C =
 \begin{pmatrix}
 A_{IJ} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}$:

In [18]:
C = np.empty((k+1, k+1))  # Prepare an empty arry of shape (k+1, k+1)
C[:-1, :-1] = A_IJ  # Assign A_IJ to the top left k x k submatrix
C[-1, :-1] = 1  # Assign 1 to the first k entries of the row at the bottom
C[:-1, -1] = -1  # Assign -1 to the first k entries of the right most column
C[-1, -1] = 0  # Assign 0 to the bottom right corner

In [19]:
C

array([[ 3.,  3., -1.],
       [ 2.,  5., -1.],
       [ 1.,  1.,  0.]])

Now solve the equation $C \begin{pmatrix} y_J \\ u \end{pmatrix} = e$:

In [20]:
sol_1 = np.linalg.solve(C, e)

In [21]:
sol_1

array([ 0.66666667,  0.33333333,  3.        ])

`sol_1[:-1]` corresponds to $y_J$, and `sol_1[-1]` to $u$:

In [22]:
y_J = sol_1[:-1]
y_J

array([ 0.66666667,  0.33333333])

In [23]:
u = sol_1[-1]
u

3.0

Check that $y_J \gg 0$:

In [24]:
(sol_1[:-1] > 0).all()

True

In [25]:
(A[[], :] <= sol_1[-1]).all()

True

One way to check that $u \geq (A_J y_J)_i$ for all $i \notin I$:

In [26]:
I_c = np.setdiff1d(M, I)  # Complement of I in M
A[I_c, :][:, J] @ y_J  # Vector given by (A_J y_J)_i for i in complement of I

array([ 2.])

In [27]:
(u >= A[I_c, :][:, J] @ y_J).all()

True

So this is a candidate for the mixed action for player 2 in an equilibrium:

In [28]:
y = np.zeros(n)
y[J] = y_J
y

array([ 0.66666667,  0.33333333])

(`y` is the same as `y_J` because $J = M$,
but let's make the code general.)

Do the same exercise for player 2:

Construct
$D =
 \begin{pmatrix}
 B'_{JI} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}$:

In [29]:
D = np.empty((k+1, k+1))  # Prepare an empty arry of shape (k+1, k+1)
D[:-1, :-1] = B_T_JI  # Assign A_IJ to the top left k x k submatrix
D[-1, :-1] = 1  # Assign 1 to the first k entries of the row at the bottom
D[:-1, -1] = -1  # Assign -1 to the first k entries of the right most column
D[-1, -1] = 0  # Assign 0 to the bottom right corner

In [30]:
D

array([[ 3.,  2., -1.],
       [ 2.,  6., -1.],
       [ 1.,  1.,  0.]])

Solve $D \begin{pmatrix} x_I \\ v \end{pmatrix} = e$:

In [31]:
sol_2 = np.linalg.solve(D, e)
sol_2

array([ 0.8,  0.2,  2.8])

Check that $x_I \gg 0$:

In [32]:
(sol_2[:-1] > 0).all()

True

Check that $v \geq (B'_I x_I)_j$ for all $j \notin J$:

In [33]:
J_c = np.setdiff1d(N, J)
(sol_2[-1] >= B[J_c, :][:, I] @ sol_2[:-1]).all()

True

So this is an equilibrium mixed action for player 1:

In [34]:
x = np.zeros(m)
x[I] = sol_2[:-1]
x

array([ 0.8,  0.2,  0. ])

We have found a Nash equilibrium:

In [35]:
(x, y)

(array([ 0.8,  0.2,  0. ]), array([ 0.66666667,  0.33333333]))

### Example: $I = \{0, 2\}$, $J = \{0, 1\}$

Let's do the same exercise for $I = \{0, 2\}$ and $J = \{0, 1\}$.

In [36]:
k = 2
I = [0, 2]
J = [0, 1]

To get the matrix `C`, let's use the previous `C`,
just replacing the submatrix in `C[:-1, :-1]`:

In [37]:
C[:-1, :-1] = A[I, :][:, J]

In [38]:
C

array([[ 3.,  3., -1.],
       [ 0.,  6., -1.],
       [ 1.,  1.,  0.]])

In [39]:
sol_1 = np.linalg.solve(C, e)

In [40]:
sol_1

array([ 0.5,  0.5,  3. ])

Check the positivity:

In [41]:
(sol_1[:-1] > 0).all()

True

Check the optimality:

In [42]:
I_c = np.setdiff1d(M, I)
(sol_1[-1] >= A[I_c, :][:, J] @ sol_1[:-1]).all()

False

This means that $v \neq \max_i (A_J y_J)_i$.

So this implies that there is no Nash equilibrium for the support pair $(I, J)$.
But let's just check also the conditions for player 2.

In [43]:
D[:-1, :-1] = B_T[J, :][:, I]

In [44]:
D

array([[ 3.,  3., -1.],
       [ 2.,  1., -1.],
       [ 1.,  1.,  0.]])

In [45]:
sol_2 = np.linalg.solve(D, e)
sol_2

array([ 2., -1.,  3.])

In [46]:
(sol_2[:-1] > 0).all()

False

This means that the solution cannot be a probability distribution.

## Wrap the procedure in a function

Let's write a function that wraps the above procedure.

The following is just one possible implementation design:

* The arguments are
  * payoff matrix `a`;
  * support `I` of the player in consideration;
  * support `J` of the opponent player;
  * array `out` that stores the candidate mixed action.
* If there is a mixed action of the opponent with support `J`
  against which the actions in `I` are best responses,
  then return `True`; otherwise return `False`.
  In the former case, the mixed action is stored in `out`.
* Array `out` must be of length equal to the number of the opponent's actions.